In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import layers, models
from sklearn.model_selection import train_test_split
from skimage.io import imread
from skimage.transform import resize

# Segmetation Model

In [2]:
import os
import numpy as np
import tensorflow as tf
from skimage.io import imread
from skimage.transform import resize
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications import VGG16
from tensorflow.keras import layers, models
from tensorflow.keras.regularizers import l2

# Function to load data
def load_data(images_path, masks_path):
    images = []
    masks = []

    # Load images
    for img_file in os.listdir(images_path):
        if img_file.lower().endswith(('.png', '.jpg', '.jpeg', '.tif', '.bmp')):
            img = imread(os.path.join(images_path, img_file))
            img = resize(img, (224, 224), mode='constant', preserve_range=True)
            if img.ndim == 2:  # If grayscale, convert to 3 channels
                img = np.stack((img,) * 3, axis=-1)
            if img.shape == (224, 224, 3):  # Ensure valid shape before appending
                images.append(img)

    # Load masks
    for mask_file in os.listdir(masks_path):
        if mask_file.lower().endswith(('.png', '.jpg', '.jpeg', '.tif', '.bmp')):
            mask = imread(os.path.join(masks_path, mask_file))
            mask = resize(mask, (224, 224), mode='constant', preserve_range=True)
            if mask.ndim == 2:  # Ensure mask has a single channel
                mask = np.expand_dims(mask, axis=-1)
            if mask.shape == (224, 224, 1):  # Ensure valid shape before appending
                masks.append(mask)

    # Debugging information
    print(f"Loaded {len(images)} images and {len(masks)} masks.")
    
    # Ensure same number of images and masks are loaded
    if len(images) != len(masks):
        print("Mismatch between number of images and masks.")
        return np.array([]), np.array([])  # Return empty arrays if mismatch occurs

    return np.array(images), np.array(masks)

# Function to preprocess data
def preprocess_data(images, masks):
    images = images.astype('float32') / 255.0  # Normalize images
    masks = masks.astype('float32') / 255.0  # Normalize masks
    masks[masks > 0] = 1  # Convert to binary
    return images, masks

# Function to split data
def split_data(images, masks):
    return train_test_split(images, masks, test_size=0.2, random_state=42)

# Function to build U-Net model with VGG16 backbone
def unet_with_backbone(input_shape=(224, 224, 3)):
    base_model = VGG16(weights='imagenet', include_top=False, input_shape=input_shape)

    # Encoder
    encoder_output = base_model.output

    # Bottleneck
    bottleneck = layers.Conv2D(1024, (3, 3), activation='relu', padding='same')(encoder_output)
    bottleneck = layers.Dropout(0.5)(bottleneck)
    bottleneck = layers.Conv2D(1024, (3, 3), activation='relu', padding='same')(bottleneck)

    # Decoder with skip connections
    up1 = layers.Conv2DTranspose(512, (2, 2), strides=(2, 2), padding='same')(bottleneck)
    up1 = layers.concatenate([up1, base_model.get_layer("block5_conv3").output])
    up1 = layers.Conv2D(512, (3, 3), activation='relu', padding='same', kernel_regularizer=l2(0.001))(up1)

    up2 = layers.Conv2DTranspose(256, (2, 2), strides=(2, 2), padding='same')(up1)
    up2 = layers.concatenate([up2, base_model.get_layer("block4_conv3").output])
    up2 = layers.Conv2D(256, (3, 3), activation='relu', padding='same', kernel_regularizer=l2(0.001))(up2)

    up3 = layers.Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(up2)
    up3 = layers.concatenate([up3, base_model.get_layer("block3_conv3").output])
    up3 = layers.Conv2D(128, (3, 3), activation='relu', padding='same', kernel_regularizer=l2(0.001))(up3)

    up4 = layers.Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(up3)
    up4 = layers.concatenate([up4, base_model.get_layer("block2_conv2").output])
    up4 = layers.Conv2D(64, (3, 3), activation='relu', padding='same', kernel_regularizer=l2(0.001))(up4)

    # Final output layer
    outputs = layers.Conv2D(1, (1, 1), activation='sigmoid')(up4)

    # Create model
    model = models.Model(inputs=[base_model.input], outputs=[outputs])
    for layer in base_model.layers:
        layer.trainable = False

    return model

# Function to train the model
def train_model(model, X_train, y_train, X_val, y_val):
    optimizer = Adam(learning_rate=0.00005)

    def dice_loss(y_true, y_pred, smooth=1e-6):
        y_true_f = tf.keras.backend.flatten(y_true)
        y_pred_f = tf.keras.backend.flatten(y_pred)
        intersection = tf.keras.backend.sum(y_true_f * y_pred_f)
        return 1 - (2. * intersection + smooth) / (tf.keras.backend.sum(y_true_f) + tf.keras.backend.sum(y_pred_f) + smooth)

    model.compile(optimizer=optimizer, loss=dice_loss, metrics=[tf.keras.metrics.MeanIoU(num_classes=2)])

    # Callbacks
    early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True, verbose=1)
    model_checkpoint = ModelCheckpoint('seg_model.keras', monitor='val_mean_io_u', save_best_only=True, mode='max', verbose=1)

    # Data augmentation
    image_datagen = ImageDataGenerator(rotation_range=20, width_shift_range=0.1, height_shift_range=0.1,
                                       shear_range=0.1, zoom_range=0.1, horizontal_flip=True, fill_mode='nearest')
    mask_datagen = ImageDataGenerator(rotation_range=20, width_shift_range=0.1, height_shift_range=0.1,
                                      shear_range=0.1, zoom_range=0.1, horizontal_flip=True, fill_mode='nearest')

    image_datagen.fit(X_train)
    mask_datagen.fit(y_train)

    # Create generators
    image_generator = image_datagen.flow(X_train, batch_size=8, seed=42)
    mask_generator = mask_datagen.flow(y_train, batch_size=8, seed=42)
    train_generator = zip(image_generator, mask_generator)

    # Train the model
    history = model.fit(train_generator, validation_data=(X_val, y_val), epochs=100, steps_per_epoch=len(X_train) // 8,
                        callbacks=[early_stopping, model_checkpoint])
    return history

# Set paths
images_path = "D:/NeuroCure-main/NeuroCure-main/dataset/images"
masks_path = "D:/NeuroCure-main/NeuroCure-main/dataset/masks"

# Load, preprocess, and split data
images, masks = load_data(images_path, masks_path)
if images.size == 0 or masks.size == 0:
    print("No images or masks found. Check directory paths.")
else:
    images, masks = preprocess_data(images, masks)
    X_train, X_val, y_train, y_val = split_data(images, masks)

    # Build, train, and evaluate model
    model = unet_with_backbone()
    history = train_model(model, X_train, y_train, X_val, y_val)


Loaded 5895 images and 2498 masks.
Mismatch between number of images and masks.
No images or masks found. Check directory paths.


# Applying mask on original Image

In [3]:
# import numpy as np
# from keras.models import load_model
# import matplotlib.pyplot as plt

# # Define the custom loss function
# def dice_loss(y_true, y_pred, smooth=1e-6, weight_bg=0.2, weight_fg=0.8):
#         y_true_f = tf.keras.backend.flatten(y_true)
#         y_pred_f = tf.keras.backend.flatten(y_pred)

#         # Compute the intersection
#         intersection = tf.keras.backend.sum(y_true_f * y_pred_f)

#         # Compute the Dice coefficient
#         dice = (2. * intersection + smooth) / (tf.keras.backend.sum(y_true_f) + 
#                                                tf.keras.backend.sum(y_pred_f) + smooth)

#         return 1 - dice

# # 1. Load the saved model with custom_objects
# model = load_model('/kaggle/input/seg/tensorflow2/default/1/seg_model2.keras', custom_objects={'dice_loss': dice_loss})

# # 2. Select the first 5 images and masks from your dataset
# X_val_5 = X_val[:5]  # Assuming X_val is your validation set of images
# y_val_5 = y_val[:5]  # Corresponding masks

# # 3. Predict masks for the first 5 images
# predicted_masks = model.predict(X_val_5)

# # 4. Visualize the results
# # Plot the first 5 images, their predicted masks, and the actual masks
# for i in range(5):
#     plt.figure(figsize=(12, 6))

#     # Display the original image
#     plt.subplot(1, 3, 1)
#     plt.imshow(X_val_5[i], cmap='gray')
#     plt.title('Original Image')

#     # Display the predicted mask
#     plt.subplot(1, 3, 2)
#     plt.imshow(np.squeeze(predicted_masks[i]), cmap='gray')  # Squeeze to remove extra dimensions
#     plt.title('Predicted Mask')

#     # Display the actual mask
#     plt.subplot(1, 3, 3)
#     plt.imshow(np.squeeze(y_val_5[i]), cmap='gray')  # Squeeze to remove extra dimensions
#     plt.title('Actual Mask')

#     plt.show()


In [4]:
# import numpy as np
# import matplotlib.pyplot as plt

# def overlay_mask_on_image(image, predicted_mask, alpha=0.5):
#     # Ensure the mask is binary
#     binary_mask = np.squeeze(predicted_mask) > 0.5  # Thresholding if needed
    
    
#     # Create a red overlay with the same shape as the original image
#     red_overlay = np.zeros_like(image)  # Create an empty array for the red overlay
#     red_overlay[binary_mask] = [255, 255, 0]  # Apply red color to the areas where the mask is True
    
#     # Blend the original image and the red overlay
#     overlay_image = np.where(binary_mask[..., None], red_overlay, image)  # Use the mask to overlay red on the original image

#     return overlay_image

# # Usage

# for i in range(0,4):
#     predicted_mask = predicted_masks[i]  # Replace with your actual predicted mask
#     overlayed_image = overlay_mask_on_image(X_val_5[i], predicted_mask)

#     # Plot the results
#     plt.figure(figsize=(5,5),)
#     plt.imshow(overlayed_image)
#     plt.axis('off')
#     plt.show()
